# Plenoptic-Toolbox To D-NeRF dataset format


#### Notes on D-NeRF dataset:

In the nerfstudio DNeRF parser, `camera_angle_x` parameter (found in the transforms files) has a $1/tanh(0.5*x)$ relationship with the focal length. While I did mess around with this in [Desmos](https://www.desmos.com/calculator/xw0lodoghb) and initially selected a high $x=4.$ value. It didn't play well with kplanes (tested with scene contraction and varying near and far plane positions). In the end $x=0.6$ worked best despite how counter intuitive this is... The parameter is important for good rendering but is not directly recoverable from the nerfstudio dataset (it might be I just haven't looked into it).

Otherwise the `rotation` parameter found for each frame doesn't seem to have any impact on performance (at least in the tests I ran with K-Planes, **this maybe different for other models!!!**).

---
# Configuration
---


In [1]:
import os

# Set the plenoptic toolbox (ptb) config and video files
config_fp ='data/plenoptic_toolbox/161029_sports1/calibration_161029_sports1.json'
video_fp = 'data/plenoptic_toolbox/161029_sports1/hdVideos/'
# Set the video type
if 'hdVideos' in video_fp:
    camType = 'hd'
elif 'vgaVideos' in video_fp:
    camType = 'vga'
else: # default currently assumes VGA camera dataset
    camType = 'vga'
# Set the output folder
output_fp = 'data/plenoptic_toolbox/161029_sports1/dnerf/'

# Assert input data exists
assert os.path.exists(config_fp), AssertionError(f'Config Non-Existent : {config_fp}')
assert os.path.exists(video_fp), AssertionError(f'Config Non-Existent : {video_fp}')


# Construct folders (replace existing folders)
os.makedirs(output_fp, exist_ok=True) # create root folder
train_im_fp = output_fp+'train/'
test_im_fp = output_fp+'test/'
val_im_fp = output_fp+'val/'
os.makedirs(train_im_fp, exist_ok=True) # create train, test and val folders
os.makedirs(test_im_fp, exist_ok=True)
os.makedirs(val_im_fp, exist_ok=True)


In [2]:
import json
import numpy as np

with open(config_fp, 'r') as fp:
    config = json.load(fp)

# Initialise used_frame dict
used_frames = {} # dict for storing "[camera]":[frame0, frame 12, frame 129, ...] (frames used in train and/or test)

# Filter through camera data and store video pose data
cameras = []
for cam in config['cameras']:
    if cam['type'] == camType:
        # Construct a 4x4 transformation matrix
        R = np.array(cam['R'])
        t = np.array(cam['t'])
        transformation_matrix = np.eye(4)
        transformation_matrix[:3, :3] = R
        transformation_matrix[:3, 3] = t.flatten()

        # Store necessary data
        cameras.append({
            "name":cam['name'],
            "transformation_matrix":transformation_matrix
        })

        # Initialise store for each camera
        used_frames[cam['name']] = []


In [3]:
# Extract training data
rotation = 0.3141592653589793
camAngleX = 0.6911112070083618

import cv2

frameNum = 0
loopflag = True # set this to a high value for now
camNum = len(cameras)

training_data = {
    "frames":[],
    "camera_angle_x": camAngleX,
}

while loopflag:
    for i, cam in enumerate(cameras):
        # Construct filepath to video
        if camType == 'hd':
            name_start = 'hd_'
        elif camType == 'vga':
            name_start = 'vga_'
        else:
            print('You need to select hd or vga selected')
            exit()
        
        fp = video_fp+name_start+cam['name']+'.mp4'
        assert os.path.exists(fp), AssertionError(f'Video Not Found: {fp}')
        
        # Get Number of cameras and number of frames
        video = cv2.VideoCapture(str(fp))

        # Get the total number of frames
        if frameNum == 0:        
            total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
        video.set(cv2.CAP_PROP_POS_FRAMES, frameNum)
        ret, frame = video.read()

        # Append data to training dict
        training_data["frames"].append(
            {
                "file_path":f'train/r_{frameNum}',
                "rotation": rotation,
                "time": float(frameNum/total_frames),
                "transform_matrix":cam["transformation_matrix"].tolist()
            }
        )
        
        
        cv2.imwrite(train_im_fp+f'r_{frameNum}.png', frame)
        used_frames[cam['name']].append(frameNum)

        frameNum += 1
        
        if frameNum == (total_frames):
            loopflag = False
            break


In [4]:
import random

# Extract test and val data
testNum = 100
valNum = 100

testing_data = {
    "frames":[],
    "camera_angle_x": camAngleX,
}
validation_data = {
    "frames":[],
    "camera_angle_x": camAngleX,
}

for j in range(testNum):
    camNum = (j % len(cameras))
    cam = cameras[camNum]
    used_cam_frames = used_frames[cam['name']]

    # Construct filepath to video
    if camType == 'hd':
        name_start = 'hd_'
    elif camType == 'vga':
        name_start = 'vga_'
    else:
        print('You need to select hd or vga selected')
        exit()
        
    fp = video_fp+name_start+cam['name']+'.mp4'
    assert os.path.exists(fp), AssertionError(f'Video Not Found: {fp}')
    
    # Get Number of cameras and number of frames
    video = cv2.VideoCapture(str(fp))
    
    # Randomly generate a frame to extract (between 0 and # Frames)
    frameNum = random.randint(0, total_frames-1)
    
    # If frame is already selected we need to generate new frame
    while frameNum in used_cam_frames:
        frameNum = random.randint(0, total_frames-1)

    used_frames[cam['name']].append(frameNum)
    video.set(cv2.CAP_PROP_POS_FRAMES, frameNum)
    ret, frame = video.read()

    # Append data to training dict
    testing_data["frames"].append(
        {
            "file_path":f'test/r_{frameNum}',
            "rotation": rotation,

            "time": float(frameNum/total_frames),
            "transform_matrix":cam["transformation_matrix"].tolist()
        }
    ) 
    cv2.imwrite(test_im_fp+f'r_{j}.png', frame)


for j in range(valNum):
    camNum = (j % len(cameras))
    cam = cameras[camNum]
    used_cam_frames = used_frames[cam['name']]

    # Construct filepath to video
    if camType == 'hd':
        name_start = 'hd_'
    elif camType == 'vga':
        name_start = 'vga_'
    else:
        print('You need to select hd or vga selected')
        exit()
        
    fp = video_fp+name_start+cam['name']+'.mp4'
    assert os.path.exists(fp), AssertionError(f'Video Not Found: {fp}')
    
    # Get Number of cameras and number of frames
    video = cv2.VideoCapture(str(fp))
    
    # Randomly generate a frame to extract (between 0 and # Frames)
    frameNum = random.randint(0, total_frames-1)
    
    # If frame is already selected we need to generate new frame
    while frameNum in used_cam_frames:
        frameNum = random.randint(0, total_frames-1)

    used_frames[cam['name']].append(frameNum)
    video.set(cv2.CAP_PROP_POS_FRAMES, frameNum)
    ret, frame = video.read()

    # Append data to training dict
    validation_data["frames"].append(
        {
            "file_path":f'val/r_{frameNum}',
            "rotation": rotation,
            "time": float(frameNum/total_frames),
            "transform_matrix":cam["transformation_matrix"].tolist()
        }
    ) 
    cv2.imwrite(val_im_fp+f'r_{j}.png', frame)

In [5]:

with open(output_fp+'transforms_train.json','w') as fp:
    json.dump(training_data, fp)
with open(output_fp+'transforms_test.json','w') as fp:
    json.dump(testing_data, fp)
with open(output_fp+'transforms_val.json','w') as fp:
    json.dump(validation_data, fp)


In [2]:
# Import
import json
from pathlib import Path
import random
import shutil

import numpy as np
from skimage.metrics import structural_similarity as ssim
from tqdm.notebook import tqdm

from utils_ import *

### TODO - think of a title

**Args:**


In [7]:

def handler(d_fp, o_fp, v_fp, img_fp, meth, rotation:float=0.0, camera_angle_x:float=0.0):
    d_fp = Path(d_fp)
    v_fp = Path(v_fp)
    img_fp = d_fp / img_fp

    transforms_fp = d_fp/'transforms.json'

    # Sanity Checks
    pathchecks([d_fp, v_fp, img_fp])
    folderchecks([d_fp, img_fp])
    
    # meth = 'linear'
    # Handle exhaustive method
    if meth == 'exhaustive':
        exhaustive(d_fp, o_fp, v_fp, img_fp,transforms_fp)    
    elif meth == 'linear':
        linear(d_fp, o_fp, v_fp, img_fp,transforms_fp, rotation=rotation, camera_angle_x=camera_angle_x)    